<a href="https://colab.research.google.com/github/TemsyChen/DS-Unit-2-Linear-Models/blob/master/TemsyChen_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [4]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

###Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [5]:
#Create a subset

df_subset = df[(df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS')&
               (df['SALE_PRICE']>=100000)&(df['SALE_PRICE']<=2000000)]


df_subset.head(5)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [6]:
#Check the values in the subset

df_subset['SALE_PRICE'].value_counts()

500000    48
550000    43
450000    40
525000    40
650000    36
          ..
371000     1
413000     1
672045     1
457000     1
248000     1
Name: SALE_PRICE, Length: 1002, dtype: int64

In [7]:
df_subset['BUILDING_CLASS_CATEGORY'].value_counts()

01 ONE FAMILY DWELLINGS    3164
Name: BUILDING_CLASS_CATEGORY, dtype: int64

### Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [8]:
df['SALE_PRICE'].dtypes

dtype('int64')

In [9]:
#Do train/test split

train = df[df['SALE_DATE']<'04/01/2019']
test = df[df['SALE_DATE']>='04/01/2019']

###Do one-hot encoding of categorical features.

In [10]:
#Explore what features are low-cardinality categories

train.describe(exclude='number')


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,18167,18167,18167,18167,18167,18167,3972,18139,18167,18167
unique,5,11,43,10,140,17926,1450,3207,140,90
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,D4,N/A ROCKAWAY BOULEVARD,4,0,D4,01/24/2019
freq,5883,15058,4094,8911,2640,5,81,5784,2640,480


In [11]:
train['BOROUGH'].value_counts()

4    5883
3    5163
1    3670
5    1733
2    1718
Name: BOROUGH, dtype: int64

In [17]:
#Visualize the distribution by borough
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x=train['BOROUGH']);
# sns.countplot(train['BOROUGH']);

In [14]:

train.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [15]:
#Exclude other non-numeric columns with high cardinality.

target = 'SALE_PRICE'
high_cardinality = ['NEIGHBORHOOD','BUILDING_CLASS_CATEGORY','TAX_CLASS_AT_PRESENT','EASE-MENT','BUILDING_CLASS_AT_PRESENT','ADDRESS',
                    'APARTMENT_NUMBER','LAND_SQUARE_FEET','BUILDING_CLASS_AT_TIME_OF_SALE','SALE_DATE']
features = train.columns.drop([target] + high_cardinality)
features

Index(['BOROUGH', 'BLOCK', 'LOT', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [16]:
train.shape, test.shape

((18167, 21), (4873, 21))

In [17]:
#Arrange train and test variables

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((18167, 10), (4873, 10), (18167,), (4873,))

In [18]:
#Do one-hot encoding

import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [19]:
X_train.shape

(18167, 14)

In [20]:
X_train.head()

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
0,1,0,0,0,0,716,1246,10011.0,1.0,0.0,1.0,1979.0,2007.0,2
1,1,0,0,0,0,812,68,10018.0,0.0,6.0,6.0,15435.0,1920.0,4
2,1,0,0,0,0,839,69,10018.0,0.0,7.0,7.0,11332.0,1930.0,4
3,1,0,0,0,0,592,1041,10014.0,1.0,0.0,1.0,500.0,0.0,2
4,1,0,0,0,0,1379,1402,10065.0,1.0,0.0,1.0,6406.0,0.0,2


In [21]:
X_train.isnull().sum()

BOROUGH_1                    0
BOROUGH_2                    0
BOROUGH_3                    0
BOROUGH_4                    0
BOROUGH_5                    0
BLOCK                        0
LOT                          0
ZIP_CODE                     0
RESIDENTIAL_UNITS            0
COMMERCIAL_UNITS             0
TOTAL_UNITS                  0
GROSS_SQUARE_FEET            0
YEAR_BUILT                   5
TAX_CLASS_AT_TIME_OF_SALE    0
dtype: int64

###Do feature selection with SelectKBest.

In [22]:
#Replace NaN's from X_train

X_train = X_train.fillna(0)
print(X_train.isnull().sum())

X_test = X_test.fillna(0)
print(X_test.isnull().sum())

BOROUGH_1                    0
BOROUGH_2                    0
BOROUGH_3                    0
BOROUGH_4                    0
BOROUGH_5                    0
BLOCK                        0
LOT                          0
ZIP_CODE                     0
RESIDENTIAL_UNITS            0
COMMERCIAL_UNITS             0
TOTAL_UNITS                  0
GROSS_SQUARE_FEET            0
YEAR_BUILT                   0
TAX_CLASS_AT_TIME_OF_SALE    0
dtype: int64
BOROUGH_1                    0
BOROUGH_2                    0
BOROUGH_3                    0
BOROUGH_4                    0
BOROUGH_5                    0
BLOCK                        0
LOT                          0
ZIP_CODE                     0
RESIDENTIAL_UNITS            0
COMMERCIAL_UNITS             0
TOTAL_UNITS                  0
GROSS_SQUARE_FEET            0
YEAR_BUILT                   0
TAX_CLASS_AT_TIME_OF_SALE    0
dtype: int64


In [23]:
#Select the 7 best features that correlate with the target

#IMPORTANT:
#.fit_transform on the train set
#.transform on the test set

from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(k=5)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

In [24]:
X_train.shape, X_train_selected.shape

((18167, 14), (18167, 5))

In [25]:
#Which features were selected?

selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
  print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
  print(name)

Features selected:
BOROUGH_1
RESIDENTIAL_UNITS
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET


Features not selected:
BOROUGH_2
BOROUGH_3
BOROUGH_4
BOROUGH_5
BLOCK
LOT
ZIP_CODE
YEAR_BUILT
TAX_CLASS_AT_TIME_OF_SALE


In [26]:
#How many features should be selected?

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
  print(f'{k} features')

  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_selected = selector.fit_transform(X_train, y_train)
  X_test_selected = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  y_pred = model.predict(X_test_selected)
  mae = mean_absolute_error(y_test, y_pred)
  print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

  #5 features has the lowest MAE

1 features
Test Mean Absolute Error: $2,095,760 

2 features
Test Mean Absolute Error: $2,097,650 

3 features
Test Mean Absolute Error: $2,078,880 

4 features
Test Mean Absolute Error: $2,055,796 

5 features
Test Mean Absolute Error: $2,051,342 

6 features
Test Mean Absolute Error: $2,069,696 

7 features
Test Mean Absolute Error: $2,073,351 

8 features
Test Mean Absolute Error: $2,073,416 

9 features
Test Mean Absolute Error: $2,092,770 

10 features
Test Mean Absolute Error: $2,093,420 

11 features
Test Mean Absolute Error: $2,114,088 

12 features
Test Mean Absolute Error: $2,114,088 

13 features
Test Mean Absolute Error: $2,123,308 

14 features
Test Mean Absolute Error: $2,126,498 



### Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

In [27]:
#Fit a multiple regression with the top 5 features according to SelectKBest

#Rearrange X matrices

features = ['BOROUGH_1','RESIDENTIAL_UNITS','COMMERCIAL_UNITS','TOTAL_UNITS','GROSS_SQUARE_FEET']

print(f'Linear Regression, dependent on: {features}')

X_train_fit = X_train[features]
X_test_fit = X_test[features]

X_train_fit

Linear Regression, dependent on: ['BOROUGH_1', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET']


,BOROUGH_1,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET
0,1,1.0,0.0,1.0,1979.0
1,1,0.0,6.0,6.0,15435.0
2,1,0.0,7.0,7.0,11332.0
3,1,1.0,0.0,1.0,500.0
4,1,1.0,0.0,1.0,6406.0
...,...,...,...,...,...
18162,0,2.0,0.0,2.0,2268.0
18163,0,1.0,0.0,1.0,1151.0
18164,0,2.0,1.0,3.0,2800.0
18165,0,1.0,0.0,1.0,1526.0


In [28]:
#Fit the model

model.fit(X_train_fit, y_train)
y_pred = model.predict(X_train_fit)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: {mae:.2f} percentage points')

Train Error: 1334267.55 percentage points


In [29]:
#Apply the model to new data

y_pred = model.predict(X_test_fit)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: {mae:.2f} percentage points')

Test Error: 2069236.72 percentage points


In [30]:
#Repeat fitting and testing the model using selectedKBest features

model.fit(X_train_selected, y_train)
y_pred = model.predict(X_train_selected)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: {mae:.2f} percentage points')

Train Error: 1388320.11 percentage points


In [31]:
y_pred = model.predict(X_test_selected)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: {mae:.2f} percentage points')

Test Error: 2126497.79 percentage points


In [32]:
#Get the coefficient and intercept

model.intercept_, model.coef_

(1289104.8200740272,
 array([ 1.33369261e+06, -9.69837386e+05, -1.41346852e+05,  1.18310094e+05,
        -3.40818464e+05, -5.08547522e+01, -1.83014763e+02, -6.93027935e+01,
         3.04347133e+05,  6.50217269e+05, -3.93075308e+05,  2.80451898e+02,
        -1.74704930e+02,  2.30916566e+05]))

In [35]:
#Do ridge regression

#Find the best alpha

from sklearn.linear_model import RidgeCV

alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

ridge = RidgeCV(alphas=alphas, normalize=True)
ridge.fit(X_train_selected, y_train)
ridge.alpha_

0.1

###Get mean absolute error for the test set.

In [50]:
#Get predictions
from sklearn.linear_model import Ridge

model = Ridge(alpha=5.0, normalize=True)
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

#Get Test MAE
mae = mean_absolute_error(y_test, y_pred)
display(f'Test Mean Absolute Error: ${mae:,.0f}')

'Test Mean Absolute Error: $1,731,436'